ARCHITECTURE:

We need data with Comment and labelled as Toxic or not, we can get it done using supervised learning.

Fr Example:

INPUTS            LABELS (we will have columns as "toxic, severe toxic, threat , racist")
i hate you        1,0,0,0
i love you        0,0,0,0
you suck          0,1,1,0

We will tokenise our string of inputs and convert to sequence of int and then into [embeddings]



DEEP LEARNING: 

We are gona use multiple LSTM layers, cause they are good for sequences of embeddings in our used case. We are then get output as 1 and 0, telling us toxicity or not.



1.INSTALL DEPENDENCIES

In [2]:
import os
import pandas as pd
import tensorflow as tf
import numpy as np


In [14]:
df = pd.read_csv('datset/train.csv')

#2 Pre-Processing


In [24]:
from tensorflow.keras.layers import TextVectorization

In [25]:
TextVectorization??

Init signature:
TextVectorization(
    max_tokens=None,
    standardize='lower_and_strip_punctuation',
    split='whitespace',
    ngrams=None,
    output_mode='int',
    output_sequence_length=None,
    pad_to_max_tokens=False,
    vocabulary=None,
    idf_weights=None,
    sparse=False,
    ragged=False,
    encoding='utf-8',
    name=None,
    **kwargs,
)
Source:        
@keras_export("keras.layers.TextVectorization")
class TextVectorization(Layer):
    """A preprocessing layer which maps text features to integer sequences.

    This layer has basic options for managing text in a Keras model. It
    transforms a batch of strings (one example = one string) into either a list
    of token indices (one example = 1D tensor of integer token indices) or a
    dense representation (one example = 1D tensor of float values representing
    data about the example's tokens). This layer is meant to handle natural
    language inputs. To handle simple string inputs (categorical strings or
    pr

In [62]:
X = df['comment_text'] #Convert X and Y values of the df
Y = df[df.columns[2:]].values



In [63]:
MAX_FEATURES = 200000 # no.of words in the vocab


In [64]:
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length = 1500,
                               output_mode='int')

In [65]:
vectorizer.adapt(X.values) #we teach our vetorizer our vocab


In [66]:
vectorizer.get_vocabulary()

['',
 '[UNK]',
 'the',
 'to',
 'of',
 'and',
 'a',
 'you',
 'i',
 'is',
 'that',
 'in',
 'it',
 'for',
 'this',
 'not',
 'on',
 'be',
 'as',
 'have',
 'are',
 'your',
 'with',
 'if',
 'article',
 'was',
 'or',
 'but',
 'page',
 'my',
 'an',
 'from',
 'by',
 'do',
 'at',
 'about',
 'me',
 'so',
 'wikipedia',
 'can',
 'what',
 'there',
 'all',
 'has',
 'will',
 'talk',
 'please',
 'would',
 'its',
 'no',
 'one',
 'just',
 'like',
 'they',
 'he',
 'dont',
 'which',
 'any',
 'been',
 'should',
 'more',
 'we',
 'some',
 'other',
 'who',
 'see',
 'here',
 'also',
 'his',
 'think',
 'im',
 'because',
 'know',
 'how',
 'am',
 'people',
 'why',
 'edit',
 'articles',
 'only',
 'out',
 'up',
 'when',
 'were',
 'use',
 'then',
 'may',
 'time',
 'did',
 'them',
 'now',
 'being',
 'their',
 'than',
 'thanks',
 'even',
 'get',
 'make',
 'good',
 'had',
 'very',
 'information',
 'does',
 'could',
 'well',
 'want',
 'such',
 'sources',
 'way',
 'name',
 'these',
 'deletion',
 'pages',
 'first',
 'help'

In [67]:
vectorized_text = vectorizer(X.values)
vectorized_text


<tf.Tensor: shape=(159571, 1500), dtype=int64, numpy=
array([[  645,    76,     2, ...,     0,     0,     0],
       [    1,    54,  2489, ...,     0,     0,     0],
       [  425,   441,    70, ...,     0,     0,     0],
       ...,
       [32445,  7392,   383, ...,     0,     0,     0],
       [    5,    12,   534, ...,     0,     0,     0],
       [    5,     8,   130, ...,     0,     0,     0]])>

In [68]:
# MCSHBAP : map , cache , shuffle , batch , prefetch , from_tensorflow_slices , list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text,Y))
dataset = dataset.cache()
dataset = dataset.shuffle(1600000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) #helps prevent bottlenecks

In [ ]:
dataset.as_numpy_iterator().next()

In [71]:
batch_X , batch_Y = dataset.as_numpy_iterator().next()

In [77]:
train = dataset.take(int(len(dataset)*.7))  #I am taking 70% of data for training
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))  #Skip first 70% then use 20% for validation
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [94]:
train_generator = train.as_numpy_iterator()
train_generator.next()

(array([[  48,   15,  720, ...,    0,    0,    0],
        [   7,  201, 6837, ...,    0,    0,    0],
        [ 480, 3286,   70, ...,    0,    0,    0],
        ...,
        [ 496,   11,   46, ...,    0,    0,    0],
        [ 165,  382,  230, ...,    0,    0,    0],
        [  47,   10,   17, ...,    0,    0,    0]]),
 array([[1, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 1, 1],
        [0, 0, 0, 0, 0, 0]]))

3. DEEP LEARNING MODEL

In [87]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM , Dropout , Bidirectional , Dense, Embedding

#Dense : Fully Connected layers
#Dropout : Regularisation

In [90]:
model = Sequential()
#Create Embedding layer , number of words + 1 (Unkown) , to create vecotr of dictionary
model.add(Embedding(MAX_FEATURES+1,32))
#using tanh cause gpu accelaration needs it to be tanh, by tensorflow.
#Bidirectional : USed for sentences, to help get context and self attention. 'I DONT HATE YOU' rather than just looking at 'hate' and just say its toxic, it will go from right to left again to look at the 'dont' word.
model.add(Bidirectional(LSTM(32,activation='tanh')))
#Feature extractor fully conected layers
model.add(Dense(128,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='relu'))
#We put 6 , as final output cause our Y has 6 values of toxic columns. So answer should be in that context only.
model.add(Dense(6,activation='sigmoid')) #used sigmoid so that answer is between 0 and 1 only

In [91]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam')

In [92]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [96]:
history = model.fit(train, epochs=1,validation_data=val)

6981/6981 ━━━━━━━━━━━━━━━━━━━━ 1820s 261ms/step - loss: 0.0820 - val_loss: 0.0501


In [ ]:
#low loss in training means predictions of training are near the actual values, but v low means overfitting

4.GET PREDICTIONS


In [111]:
input_text = vectorizer("You freaking suck, ")
input_text


<tf.Tensor: shape=(1500,), dtype=int64, numpy=array([   7, 7158,  397, ...,    0,    0,    0])>

In [112]:
res = model.predict(np.expand_dims(input_text,0))
res

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step


array([[0.9967848 , 0.30112734, 0.95979   , 0.03357739, 0.8519327 ,
        0.17071481]], dtype=float32)

In [115]:
#Use a batch of test
batch_X , batch_Y = test.as_numpy_iterator().next()
(model.predict(batch_X) > 0.5).astype(int)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


array([[0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

5.Evaluation

In [116]:
from tensorflow.keras.metrics import Precision , Recall , CategoricalAccuracy

In [117]:
pre = Precision()
re = Recall()
acc = CategoricalAccuracy()



In [118]:
for batch in test.as_numpy_iterator():
    #unpack batch
    X_true , y_true = batch
    #make prediction
    ypred = model.predict(X_true)

    #Flatten the prediction, converts into one big array than multiple array's
    y_true = y_true.flatten()
    ypred = ypred.flatten()

    pre.update_state(y_true, ypred)
    re.update_state(y_true, ypred)
    acc.update_state(y_true, ypred)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━

2024-06-11 17:00:48.859238: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [121]:
print(f'Precision: {pre.result().numpy()},Recall: {re.result().numpy()},Accuracy: {acc.result().numpy()},')

Precision: 0.7845161557197571,Recall: 0.697447657585144,Accuracy: 0.4914744198322296,


In [122]:
#WE can imporove our numbers, with the number of epoch being increased. easily


In [140]:
import tensorflow as tf
import gradio as gr

model.save('toxicity.h5')
model = tf.keras.models.load_model('toxicity.h5')

def score_comment(comment):
    vectorized_comment = vectorizer([comment])
    results = model.predict(vectorized_comment)
    
    text = ''
    for idx, col in enumerate(df.columns[2:]):
        text += '{}: {}\n'.format(col, results[0][idx]>0.5)
    
    return text


In [141]:
interface = gr.Interface(fn=score_comment, inputs=["text"] ,outputs='text')


In [142]:
interface.launch(share=True)

Running on local URL:  http://127.0.0.1:7861
Running on public URL: https://e7521e7b158e17d256.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 279ms/step
